### Naive Bayes Algoritması ile E-mail Spam Kontrolü

In [1]:
import pandas as pd

#### Veri Ön İşleme

In [2]:
# Stop Word 
# Stem işlemi vb

In [10]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [11]:
df = df[['label_num', 'text']]
df.head()

,label_num,text
0,0,Subject: enron methanol ; meter # : 988291\r\n...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,1,"Subject: photoshop , windows , office . cheap ..."
4,0,Subject: re : indian springs\r\nthis deal is t...


In [29]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------- ----- 1.3/1.5 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 13.2 MB/s eta 0:00:00


In [34]:
# Running,
# running
# runNing
# RUNNING!
# run

# RegularExpression
import re
import string
from nltk.corpus import stopwords

# stem işlemi -> kelimelerin köküne indirme. -> Running -> Run
# stopword -> the, is, at
nltk.download('stopwords')

def clean_text(text):
    text = text.lower() # tüm harfleri küçük hale getirdim
    
    #text = re.sub(r'\d+', '', text) # metindeki tüm numerik değerleri kaldırdım
    
    text = text.translate(str.maketrans('','', string.punctuation)) # metindeki bütün noktalama işaretlerini kaldırır.
    
    words = text.split() # Merhaba Dünya => [Merhaba,Dünya]
    
    words = [ word for word in words if not word.isdigit() ] # Kelimelerin yalnızca numerik olanlarını ele.

    words = [ word for word in words if word not in stopwords.words('english')  ]
    
    return words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [35]:
# Subject: hpl nom for january 9 , 2001\r\n( see
# subject hpl nom for january see
clean_text("The Subject: hpl nom for january 9 , 2001 ( see ) p2p 4.5g 3d")

['subject', 'hpl', 'nom', 'january', 'see', 'p2p', '45g', '3d']

In [ ]:
# Ben yazılım mühendisiyim
# Yazılım ben mühendisiyim